In [1]:
import os
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl

import pickle

In [2]:
Map = geemap.Map(center=[-5.4083790576723585, 105.25749076790254], zoom=8, add_google_map=False)
Map.add_basemap('HYBRID')

In [3]:
Map

Map(center=[-5.4083790576723585, 105.25749076790254], controls=(WidgetControl(options=['position', 'transparen…

In [4]:
with open('trained_v1.pickle', 'rb') as f:
    model = pickle.load(f)

In [5]:
# Designe interactive widgets


style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

aoi_widget = widgets.Checkbox(
    value=False,
    description='Use user-drawn AOI',
    style=style
)


year_widget = widgets.IntSlider(min=1984, max=2021, value=2010, description='Selected year:', width=400, style=style)

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

#labels
step1 = widgets.Label('Step 1: Select the date')
step2 = widgets.Label('Step 2: Select Region from the map')

"""
full_widget = widgets.HBox([
    widgets.VBox([step1, dateBox]),
    widgets.VBox([step2, submit])
])
"""

full_widget = widgets.VBox([
    widgets.HBox([aoi_widget, year_widget]),
    submit
])

full_widget

In [6]:
def submit_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:
            selected_year = year_widget.value
            use_aoi = aoi_widget.value
                       
            if use_aoi:
                if Map.user_roi is not None:
                    roi = Map.user_roi
                    layer_name = 'User drawn AOI'
                    geom = roi
                else:
                    output_widget.clear_output() 
                    print('No user AOI could be found.')
                    return
            else:
                print('No user AOI could be found.')
                return
               
            image = (ee.ImageCollection('COPERNICUS/S2_SR')
                     .filterBounds(Map.user_rois.geometry())
                     .filterDate('2019-09-01', '2019-12-31')
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))
                     .median().clip(Map.user_rois.geometry())
                     )

            # Add the image layer to the map and display it.
            Map.add_ee_layer(
                image, {'bands': ['B4', 'B3', 'B2'],
                        'min': 0, 'max': 2000}, 'sentinel')
            
            bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

            result = image.select(bands).classify(model)

            # # Display the clusters with the correspondent colors.
            class_values = [0,1,2,3,4,5,6]
            class_palette = ['3388ff', 'e72a08', '3ba702', 'e4f406', '44ff00','c88d0e','bab7b0']
            landcover = result.set('classification_class_values', class_values)
            landcover = landcover.set('classification_class_palette', class_palette)

            Map.addLayer(landcover, {}, 'classified')

            legend_dict = {
                'Water': '3388ff',
                'Bare soil': 'e72a08',
                'Forest': '3ba702',
                'Annual crop': 'e4f406',
                'Perennial crop': '44ff00',
                'Pasture': 'c88d0e',
                'Urban area': 'bab7b0'}
        
            Map.add_legend(legend_dict=legend_dict)


        except Exception as e:
            print(e)
            print('An error occurred during computation.')        

        Map.default_style = {'cursor': 'default'}

submit.on_click(submit_clicked)